In [ ]:
0- To run this project, you will need to install packages listed in requirement.exe file. Python version used for this project is 3.11
There is an exception for synthetic data generation. Since the latest version of repryerson repository could not been installed on this version of python, synthetic data generation has been executed using python 3.9 .
So, to start with, we need all datasets ready. dataset preprocessing has been run and all datasets have been loaded and saved in dpnice\datasets\.
To generate synthetic data, you need to run generate_synth.py from util.data.
The version of synthetic data that we have used, is uploaded in the dpnice.datasets folder, so it is possible to skip synth_data generation.

1- The first step of the project, is training the models.
for each dataset (adult, hospital, inform) we trained 3 models: RF, NN, SVM
for each model type 5 seeds have been set. to train one model for one dataset or one seed you just need to run train_models.py with desired values.
one example to train an RF model for adult dataset with rseed 2 is provided.

In [ ]:
!python train_models_optimized.py --dataset adult --rseed 2 --model RF

Note that datasets analysis is done in this step. Therefore, to make synthetic_datasets ready for next step of CF generation, it is nessecary to run train_models_optimized.py for synthetic datasets as well.
required script to train all models for all datasets is provided in folder scripts.
After training models, in dpnice folder you will have 2 new folders: dataset_loaded and pretrained, including .pkl files of processed datasets and trained models.

2- Now that all models and datasets are ready, you can generate counterfactuals by calling CF_Generation_optimized.py. This file is written using parallel programming to make the execution faster. Yet it works in regular settings with no parallelization. This function recieves dataset_name, model, CF_method, epsilon and n_count as input. Except for 'inline_LDP' and'zerocost_DP_CF' methods, n_count should be set to 3 (it is for processing only, recieving 3 and any of those methods name will result in running NICE as basis counterfactual mechanism). 
here is needed information for parameter values:


In [ ]:
datasets = ['compas','adult','default_credit']  
models = ['NN','RF','SVM']
CF_method_list = ['NICE','LDP_CF','LDP_SRR','LDP_Noisy_max','inline_LDP','synth_dp_cf','ldp_ds_cf']
# pre-processed methods: 'synth_dp_cf' (uses MST generated dataset), 'ldp_ds_cf' (uses dataset generated using laplacian noise and RR)
# in-processed methods: 'inline_LDP'
# post-processed methods: 'LDP_CF' (laplacian noise and RR),'LDP_SRR' (feature based exponential mechanism),'LDP_Noisy_max' (noisy max)
n_count_list = [0,3,5,10,20]
epsilons = [0.01,0.1,1,5,10]

The script to run this function is provided in the Scripts folder. an example for running it for one setting is provided below:
each execution, will generate 100 counterfactuals. Required information related to these counterfactuals are saved in a CSV file, in dpnice.cf_result folder, in related pile_path based on the dataset, model and seed.

In [ ]:
!python CF_generation_optimized.py --dataset adult --rseed 2 --model RF --CF_method LDP_SRR --epsilon 1 --n_count 3 

After generating all counterfactuals, we can do analysis for them. First, CF_genetation_result_analysis_optimized.py iterates over all CF_results files and extract information and saves them in a ready to analysis .pkl file in dpnice.optimized.analysis folder.

In [ ]:
!python CF_genetation_result_analysis_optimized.py

Now, running CF_genetation_visualize_results_optimized does statistics analysis over results of all counterfactuals and generates privacy-utility trade-off heatmaps over each dataset/model/seed/cf_method in related path in dpnice.optimized.visualized

In [ ]:
!python CF_genetation_visualize_results_optimized.py

To access this trade-offs aggregated over all 5 executions (5 random seeds that have been used for splitting datasets), two more steps should be passed. related commands are provided below:

In [ ]:
!python Aggregation.py
!python Aggregated_trade-offs.py

By the end of this steps, aggregated analysis over all existing settings will be accessible in dpnice/optimized/visualized/dataset/model/cf_method. Each folder includes 2 files. resuls.tex which provides the tradeoffs for NICE and the related model. The second file is result_improvement.tex which shows the improvement at each utility/privacy metrics compared to NICE.
